<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Datos

Los datos fueron obtenidos del siguiente [link](https://github.com/seftimie/TensorFlowJourney/blob/master/demos/rosalia.txt).
El archivo contiene de letras de canciones de los discos _El mal querer_ y _Los angeles_ de Rosalía.

In [80]:
# !pip install gensim

In [81]:
# !pip install tensorflow

In [2]:
# preprocessing
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import plotly.graph_objects as go
import plotly.express as px

# nlp
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [3]:
df = pd.read_csv('/content/rosalia.txt', sep='/n', header=None)
df.head()

<ipython-input-3-ebd0459ec959>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('/content/rosalia.txt', sep='/n', header=None)


,0
0,Ese cristalito roto yo sentí cómo crujía (Hm)
1,Antes de caerse al suelo ya sabía que se rompí...
2,Está parpadeando la luz del descansillo
3,"(Hm) Una voz en la escalera, alguien cruzando ..."
4,"Malamente (Eso e', así sí)"


In [4]:
df.shape

(1502, 1)

In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1502


### Preprocesamiento

In [6]:
sentence_tokens = []

for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [7]:
sentence_tokens

[['ese', 'cristalito', 'roto', 'yo', 'sentí', 'cómo', 'crujía', 'hm'],
 ['antes',
  'de',
  'caerse',
  'al',
  'suelo',
  'ya',
  'sabía',
  'que',
  'se',
  'rompía',
  'uh'],
 ['está', 'parpadeando', 'la', 'luz', 'del', 'descansillo'],
 ['hm',
  'una',
  'voz',
  'en',
  'la',
  'escalera',
  'alguien',
  'cruzando',
  'el',
  'pasillo'],
 ['malamente', 'eso', "e'", 'así', 'sí'],
 ['malamente', '¡tra', 'tra'],
 ['mal', "mu'", 'mal', "mu'", 'mal', "mu'", 'mal', "mu'", 'mal', 'mira'],
 ['malamente', 'ahh', 'ah', 'ah', 'ah', 'hm'],
 ['toma', 'que', 'toma'],
 ['está', 'en', 'la', 'mente', 'eso', "e'", '¡illo'],
 ['ay', 'malamente'],
 ['mal', "mu'", 'mal', "mu'", 'mal', "mu'", 'mal', "mu'", 'mal', 'no'],
 ['malamente', 'uh'],
 ['se',
  'ha',
  'puesto',
  'la',
  'noche',
  'rara',
  'han',
  "salí'o",
  'luna',
  'y',
  'estrellas',
  'eh'],
 ['me',
  'lo',
  'dijo',
  'esa',
  'gitana',
  '¿qué',
  'mejor',
  'no',
  'salir',
  'a',
  'verla',
  'no'],
 ['sueño',
  'que',
  'estoy',
  

In [8]:
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

### Modelo

In [36]:
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [37]:
w2v_model.build_vocab(sentence_tokens)

In [38]:
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1502


In [39]:
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 241


In [40]:
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 43500.98046875
Loss after epoch 1: 19166.01171875
Loss after epoch 2: 17208.0546875
Loss after epoch 3: 16527.8125
Loss after epoch 4: 17162.421875
Loss after epoch 5: 16343.4765625
Loss after epoch 6: 16613.5703125
Loss after epoch 7: 16536.796875
Loss after epoch 8: 16681.484375
Loss after epoch 9: 16333.984375
Loss after epoch 10: 16253.59375
Loss after epoch 11: 15554.671875
Loss after epoch 12: 16318.484375
Loss after epoch 13: 15269.65625
Loss after epoch 14: 15706.40625
Loss after epoch 15: 15777.125
Loss after epoch 16: 15370.0
Loss after epoch 17: 15880.5
Loss after epoch 18: 15394.0625
Loss after epoch 19: 15727.75


(59864, 158720)

### Comparaciones

**Comparación a través de vectores**

In [41]:
vector_quiero = w2v_model.wv.get_vector("quiero")
print(vector_quiero)

[ 1.19251132e-01  9.33083594e-02  2.68750004e-02 -3.52353416e-02
  8.89890119e-02 -1.20076776e-01  1.24419965e-01  1.64066508e-01
  3.25443852e-03 -8.37885588e-02  6.00014697e-04 -2.16480181e-01
 -2.66969632e-02 -5.37299402e-02 -1.28768533e-01  5.90159893e-02
  8.42896849e-03 -4.21986617e-02 -8.74443576e-02 -1.90021455e-01
  6.98120473e-03 -2.48617511e-02 -1.02296732e-01  1.02347564e-02
  2.88003893e-03 -1.07224971e-01 -1.04287915e-01 -9.12328884e-02
 -1.97014045e-02 -3.69618163e-02  6.71452880e-02 -1.17943123e-01
 -1.86666343e-02 -4.13259752e-02 -4.50104997e-02 -3.39578725e-02
 -4.04040292e-02 -1.31852359e-01  1.02635801e-01  5.81544116e-02
  1.52148791e-02  1.54805571e-01 -7.53295273e-02 -9.66339186e-02
  7.47956783e-02  1.31021202e-01  9.06515718e-02  1.16738789e-01
  9.46010351e-02  1.27948165e-01  1.59767479e-01 -1.24195367e-01
  3.10308603e-03  6.23522699e-02 -9.47769061e-02  1.10721335e-01
 -2.42590025e-01 -1.37356808e-02  6.10663444e-02 -2.41962466e-02
  5.70128560e-02 -2.62909

Palabras que más se relacionan

In [42]:
w2v_model.wv.most_similar(vector_quiero)

[('quiero', 1.0),
 ('lo', 0.9979907274246216),
 ('querer', 0.9976432919502258),
 ('tengo', 0.9975541830062866),
 ('vas', 0.9971269369125366),
 ('vayas', 0.996992290019989),
 ('contigo', 0.9969513416290283),
 ('ti', 0.9969386458396912),
 ('porque', 0.9968414306640625),
 ('te', 0.9967107176780701)]

Palabras que menos se relacionan

In [43]:
w2v_model.wv.most_similar(negative=[vector_quiero])

[('palmas', -0.6266674995422363),
 ('junta', -0.6433519721031189),
 ("mirá'", -0.655308187007904),
 ('ah', -0.6588268876075745),
 ('separa', -0.6621498465538025),
 ("mu'", -0.6672031879425049),
 ('las', -0.6695500016212463),
 ('canastera', -0.67014479637146),
 ('mal', -0.6724904775619507),
 ("clavá'", -0.7017909288406372)]

**Análisis de una lista de palabras**

In [44]:
palabras = ["quiero", "amor", "alma", "dolor", "sol"]

In [45]:
similar_words = {}

for palabra in palabras:
    try:
        similar_words[palabra] = w2v_model.wv.most_similar(positive=[palabra], topn=5)
    except KeyError:
        similar_words[palabra] = "Palabra no encontrada en el vocabulario."

print(f'TOP 5 de palabras más similares a las palabras\n {palabras}')
for palabra, similitudes in similar_words.items():
    print(f"\n'{palabra}':")
    if isinstance(similitudes, str):
        print(similitudes)
    else:
        for similar, score in similitudes:
            print(f"  - {similar} ({score:.4f})")


TOP 5 de palabras más similares a las palabras
 ['quiero', 'amor', 'alma', 'dolor', 'sol']

'quiero':
  - lo (0.9980)
  - querer (0.9976)
  - tengo (0.9976)
  - vas (0.9971)
  - vayas (0.9970)

'amor':
  - canta (0.9908)
  - cuerpo (0.9906)
  - ay (0.9906)
  - tarara (0.9901)
  - mío (0.9900)

'alma':
  - vía (0.9989)
  - bajo (0.9988)
  - aunque (0.9987)
  - niño (0.9987)
  - mujer (0.9987)

'dolor':
  - fin (0.9980)
  - bien (0.9973)
  - sé (0.9970)
  - pena (0.9969)
  - para (0.9968)

'sol':
  - ha (0.9983)
  - luz (0.9982)
  - día (0.9980)
  - se (0.9980)
  - dios (0.9978)


In [46]:
pares = [("mujer", "niña"), ("sol", "día"), ("amor", "querer")]

print('Similitudes entre pares\n')
for w1, w2 in pares:
    try:
        similarity = w2v_model.wv.similarity(w1, w2)
        print(f"- '{w1}' y '{w2}': {similarity:.4f}")
    except KeyError:
        print(f"One of '{w1}' or '{w2}' is not in vocabulary.")

Similitudes entre pares

- 'mujer' y 'niña': 0.9990
- 'sol' y 'día': 0.9980
- 'amor' y 'querer': 0.9871


### Visualizaciones

In [20]:
def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [23]:
vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.write_image("tsne_2d.png")
fig.show(renderer="colab") # esto para plotly en colab


Gráfico 2D disponible en el siguiente [link](https://github.com/qagustina/ceia-uba/blob/main/PLN1/tsne_2d.png).

In [24]:
vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.write_image("tsne_3d.svg")
fig.show(renderer="colab") # esto para plotly en colab


### Conclusiones
---

Se puede ver que el modelo, en general, relaciona correctamente las palabras en este contexto, se tomaron algunos fragmentos para comprobar los resultados de similaridad de la palabra _sol_ :

[...]
Que salga el sol o que no salga,

Eso qué me importa a mí,

Si la **luz** que a mí m'alumbra

Es cuando te veo a ti.


[...]
Salga el sol y venga el **día**

Que alumbre mi oscuridad

La luna me trae las penas

Que no quiero recordar

[...]
En abril por el mar

mi barca velera viene y va,

y en el horizonte

el sol se va

y un rayo de **luz**

me alumbrará.

[...]
Con el mayor enemigo del mundo

Que mira si tu pena es grande

Que era como la **luz** del sol,

Tu mirá se clava en mí.


Se utilizaron letras de canciones de una banda para generar embeddings, por lo tanto cabe aclarar que los vectores tienen la forma en función de como la banda utilizó las palabras en sus canciones.

En cuanto a la pérdida del modelo el valor inicial es muy alto pero disminuye progresivamente hasta la época 10, luego la pérdida oscila entre 15200 y 15800 sin una mejora significativa, esto puede deberse a la cantidad de datos disponibles, se podría mejorar agregando datos y repetir el entrenamiento.